<h1>PDIoT Data Analysis</h1>
<p>Hopefully by now you have collected some HAR data. We are asking you to collect data from two sensors - the Respeck (25Hz, accel and gyro) and the Thingy (25Hz, accel, gyro and magnetometer).</p>

<p> The Respeck is worn on the lower left ribcage, and the Thingy is worn in the front right pocket of the trousers. </p>

<p> We will explore some example data in this notebook. </p>

<h3>Accelerometer</h3>
<ul>
    <li>Measures acceleration (including gravity)</li>
    <li>Observing the change in direction of gravity often more useful than linear acceleration due to movement</li>
    <li>Sensor values given in g along the axis of interest</li>
    <li>Placing our sensor flat on the table should give -1g on the Z axis and 0g on the other axes</li>
    <li>Cheap to buy and low power consumption</li>
</ul>

<h3>Gyroscope</h3>
<ul>
    <li>Measures angular velocity</li>
    <li>Sensor values given in radians per second (deg/sec) along the axis of interest</li>
    <li>Placing our sensor flat on the table should give 0 values along all axes</li>
    <li>Higher power consumption</li>
</ul>

<h2>Human Activity Recognition</h2>

<p>Your are expected to research and develop the HAR algorithm yourselves during the course. A useful first stage  will be to look at some activity data visually to understand how the sensors react to different types of movement.</p>

<p>You are free to use any programming language for the data analysis part of the project, but we recommend using Python and Jupyter Notebook to quickly explore ideas. Below is a simple example using Python/Pandas to graph acceleration data from a sample of walking data.</p>

#### Basic imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple
# %matplotlib notebook

### Reading the header

The files contain a header of size 5. This is where we specify the recording values:
- sensor type (Respeck or Thingy)
- activity type
- activity code (you can find the mapping between activities and their codes in the Constants file on the app)
- subject ID (always a student number)
- notes (can be empty)

In [2]:
#filename_respeck = "./Data/Respeck_s1724067_Lying down left_03-10-2021_16-31-15.csv"
#filename_thingy = "./Data/Thingy_s1724607_Lying down left_06-10-2021_20-17-30.csv"

In [3]:


header_size = 5

with open(filename_respeck) as f:
    head = [next(f).rstrip().split('# ')[1] for x in range(header_size)]
    for l in head:
        print(l)
        

NameError: name 'filename_respeck' is not defined

### Getting the recording metadata

It's useful to store the metadata about each recording, as you will need it for later.

In [ ]:
sensor_type = ""
activity_type = ""
activity_code = -1
subject_id = ""
notes = ""

with open(filename_respeck) as f:
    head = [next(f).rstrip().split('# ')[1] for x in range(header_size)]
    for l in head:
        print(l)
        
        title, value = l.split(":")
        
        if title == "Sensor type":
            sensor_type = value.strip()
        elif title == "Activity type":
            activity_type = value.strip()
        elif title == "Activity code":
            activity_code = int(value.strip())
        elif title == "Subject id":
            subject_id = value.strip()
        elif title == "Notes":
            notes = value.strip()

You might use this later so you can pack it up into a function

In [ ]:
def extract_header_info(filename: str, header_size: int = 5) -> Tuple[str, str, int, str, str]:
    """
    :param filename: Path to recording file.
    :param header_size: The size of the header, defaults to 5.
    :returns: A 5-tuple containing the sensor type, activity type, activity code, subject id and any notes.
    """
    sensor_type = ""
    activity_type = ""
    activity_code = -1
    subject_id = ""
    notes = ""

    with open(filename) as f:
        head = [next(f).rstrip().split('# ')[1] for x in range(header_size)]
        for l in head:
            print(l)

            title, value = l.split(":")

            if title == "Sensor type":
                sensor_type = value.strip()
            elif title == "Activity type":
                activity_type = value.strip()
            elif title == "Activity code":
                activity_code = int(value.strip())
            elif title == "Subject id":
                subject_id = value.strip()
            elif title == "Notes":
                notes = value.strip()
    
    return sensor_type, activity_type, activity_code, subject_id, notes

And now we can get the variables by applying the function

In [ ]:
sensor_type, activity_type, activity_code, subject_id, notes = extract_header_info(filename=filename_respeck)

### Reading the file

You can load the file itself using Pandas. You need to specify the amount of rows to be skipped in the beginning (the header size).

In [ ]:
df_respeck = pd.read_csv(filename_respeck, header=header_size)
print(df_respeck)

To save the recording metadata for later we can append them as values in new columns

In [ ]:
df_respeck['sensor_type'] = sensor_type
df_respeck['activity_type'] = activity_type
df_respeck['activity_code'] = activity_code
df_respeck['subject_id'] = subject_id
df_respeck['notes'] = notes

In [ ]:
df_respeck

One more important value to save for later is a recording ID. This will be used to split the entire dataset into separate recordings before you start doing any further splitting into windows. The name of the file can act as the unique recording ID for each recording. 

In [ ]:
filename_respeck.split("/")[-1].split(".")[0]

In [ ]:
df_respeck['recording_id'] = filename_respeck.split("/")[-1].split(".")[0]

In [ ]:
df_respeck

One useful function is checking the frequency of your recordings. The sensors are both running at 25Hz but it is possible that some packets are dropped along the way. You can use the below function to quickly check the frequency of any of your recordings.

In [ ]:
def get_frequency(dataframe: pd.DataFrame, ts_column: str = 'timestamp') -> float:
    """
    :param dataframe: Dataframe containing sensor data. It needs to have a 'timestamp' column.
    :param ts_column: The name of the column containing the timestamps. Default is 'timestamp'.
    :returns: Frequency in Hz (samples per second)
    """

    return len(dataframe) / ((dataframe[ts_column].iloc[-1] - dataframe[ts_column].iloc[0]) / 1000)

In [ ]:
df_respeck_frequency=get_frequency(df_respeck)
df_respeck_frequency

Here we can see that the frequency of this recording is a bit over 25Hz, which is considered normal. You should be worried if your recordings deviate with more than 2Hz from the 25Hz threshold. 

You can load the thingy data in a similar way

In [ ]:

# extract header information
sensor_type, activity_type, activity_code, subject_id, notes = extract_header_info(filename=filename_thingy)

# load data
df_thingy = pd.read_csv(filename_thingy, header=header_size)

# append recording metadata
df_thingy['sensor_type'] = sensor_type
df_thingy['activity_type'] = activity_type
df_thingy['activity_code'] = activity_code
df_thingy['subject_id'] = subject_id
df_thingy['notes'] = notes

# get the recording ID
df_thingy['recording_id'] = filename_thingy.split("/")[-1].split(".")[0]

In [ ]:
df_thingy

### Visualising data

Next we will learn how to visualise the data from both sensors.

Be careful when plotting sensor data, if you are trying to compare activities you need to make sure that the axes match. Accelerometer and Gyroscope data are measured on very different scales - accelerometer data is usually in the range [-4, 4], while gyroscope data can get to the 10s and 100s. You should not plot them on the same plot.

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 12))

# plot respeck
ax[0].plot(df_respeck['accel_x'], label="accel_x")
ax[0].plot(df_respeck['accel_y'], label="accel_y")
ax[0].plot(df_respeck['accel_z'], label="accel_z")
ax[0].legend()

ax[0].set_title(f"{df_respeck['sensor_type'].values[0]} - {df_respeck['activity_type'].values[0]} \n Accelerometer data")

ax[1].plot(df_respeck['gyro_x'], label="gyro_x")
ax[1].plot(df_respeck['gyro_y'], label="gyro_y")
ax[1].plot(df_respeck['gyro_z'], label="gyro_z")
ax[1].legend()

ax[1].set_title(f"{df_respeck['sensor_type'].values[0]} - {df_respeck['activity_type'].values[0]} \n Gyroscope data")

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(12, 18))

# plot thingy
ax[0].plot(df_thingy['accel_x'], label="accel_x")
ax[0].plot(df_thingy['accel_y'], label="accel_y")
ax[0].plot(df_thingy['accel_z'], label="accel_z")
ax[0].legend()

ax[0].set_title(f"{df_thingy['sensor_type'].values[0]} - {df_thingy['activity_type'].values[0]} \n Accelerometer data")

ax[1].plot(df_thingy['gyro_x'], label="gyro_x")
ax[1].plot(df_thingy['gyro_y'], label="gyro_y")
ax[1].plot(df_thingy['gyro_z'], label="gyro_z")
ax[1].legend()

ax[1].set_title(f"{df_thingy['sensor_type'].values[0]} - {df_thingy['activity_type'].values[0]} \n Gyroscope data")

ax[2].plot(df_thingy['mag_x'], label="mag_x")
ax[2].plot(df_thingy['mag_y'], label="mag_y")
ax[2].plot(df_thingy['mag_z'], label="mag_z")
ax[2].legend()

ax[2].set_title(f"{df_thingy['sensor_type'].values[0]} - {df_thingy['activity_type'].values[0]} \n Magnetometer data")

Begin by visually inspecting a set of different activities for both sensors, to see how they might best be differentiated. Then you can begin to analyse windows of data for the signal and try to categorise it into the different activities.

## Trimming and cleaning data

You need to visually verify all of your recordings and make sure the activity starts at the very beginning of the recording and stops at the very end. 

For example, in the above Thingy recording of walking, you can see that the first 25 datapoints (1 second) were rather still, and the real walking begins a second later. You can amend this by trimming the recording to remove the first and last seconds, or however much time you think would work. 

The total length of your **trimmed and cleaned** recordings should be **30 seconds**.

To check the time of a recording (in seconds) you can run the following code:

In [ ]:
len(df_thingy)

In [ ]:
len(df_respeck)

In [ ]:
def data_is_25hz(df_respeck,df_thingy):
    get_frequency_respeck=get_frequency(df_respeck)
    get_frequency_thingy = get_frequency(df_thingy)
    if 24<get_frequency_respeck<26 and  24<get_frequency_thingy<26:
        return 'No Problem',get_frequency_respeck,get_frequency_thingy
    else:
        return 'frequency not around 25hz!!!',get_frequency_respeck,get_frequency_thingy
data_is_25hz(df_respeck,df_thingy)

In [ ]:
def found_peak(df):
    peak_index = 0
    peak_value = max(abs(df['gyro_x'])+abs(df['gyro_y'])+abs(df['gyro_z']))
    for i,row in df.iterrows():
        if abs(row['gyro_x'])+abs(row['gyro_y'])+abs(row['gyro_z']) == peak_value:
            return i

In [ ]:
get_frequency_thingy = get_frequency(df_thingy)
get_frequency_respeck=get_frequency(df_respeck)
def Get3sData(df_respeck,df_thingy,get_frequency_respeck,get_frequency_thingy):
    record_needed_respeck = get_frequency_respeck*3
    record_needed_thingy = get_frequency_thingy*3

    respeck_front = len(df_respeck)//2 - record_needed_respeck//2
    respeck_after = len(df_respeck)//2 + record_needed_respeck//2
    thingy_front = len(df_thingy)//2 - record_needed_thingy//2
    thingy_after = len(df_thingy)//2 + record_needed_thingy//2
    df_respeck_trimmed = df_respeck.iloc[int(respeck_front) :int(respeck_after)]
    df_thingy_trimmed = df_thingy.iloc[int(thingy_front):int(thingy_after)]
    return df_respeck_trimmed,df_thingy_trimmed


In [ ]:
def Get1sData(df_respeck,df_thingy,get_frequency_respeck,get_frequency_thingy):
    record_needed_respeck = get_frequency_respeck+1
    record_needed_thingy = get_frequency_thingy+1

    respeck_front = found_peak(df_respeck) - record_needed_respeck//2+5
    respeck_after = found_peak(df_respeck) + record_needed_respeck//2+5
    thingy_front = found_peak(df_thingy) - record_needed_thingy//2+5
    thingy_after = found_peak(df_thingy) + record_needed_thingy//2+5
    df_respeck_trimmed = df_respeck.iloc[int(respeck_front) :int(respeck_after)]
    df_thingy_trimmed = df_thingy.iloc[int(thingy_front):int(thingy_after)]
    return df_respeck_trimmed,df_thingy_trimmed
df_respeck_trimmed, df_thingy_trimmed = Get1sData(df_respeck,df_thingy,get_frequency_respeck,get_frequency_thingy)

In [ ]:
#found_peak(df_respeck)

In [ ]:
df_respeck_period=len(df_respeck_trimmed) / get_frequency(df_respeck_trimmed)
df_respeck_period

In [ ]:
def data_is_30s(df_respeck_trimmed,df_thingy_trimmed):
    df_respeck_period=len(df_respeck_trimmed) / get_frequency(df_respeck_trimmed)
    df_thingy_period=len(df_thingy_trimmed) / get_frequency(df_thingy_trimmed)
    if 29<df_respeck_period<31 and  29<df_thingy_period<31:
        return 'No Problem',df_respeck_period,df_thingy_period
    else:
        return 'data not around 30s!!!',df_respeck_period,df_thingy_period

In [ ]:
data_is_30s(df_respeck_trimmed,df_thingy_trimmed)

In [ ]:
df_respeck_trimmed

In [ ]:
df_thingy_trimmed

In [ ]:
def outliner_replacement(arr):
    dic={}
    outliner_list=[]
    new_list=[]
    elements = np.array(arr)
    mean = np.mean(elements, axis=0)
    sd = np.std(elements, axis=0)
    for x in arr:
        if(x < mean - 3 * sd) or (x > mean + 3 * sd):
            outliner_list.append(x)
            x=mean
            new_list.append(x)
        else:
            new_list.append(x)
    for x in outliner_list:
        dic[x]=mean
    arr = arr.replace(dic)
    return arr

In [ ]:
#df_respeck_trimmed['gyro_x'] = outliner_replacement(df_respeck_trimmed['gyro_x'])
#df_respeck_trimmed['gyro_y'] = outliner_replacement(df_respeck_trimmed['gyro_y'])
#df_respeck_trimmed['gyro_z'] = outliner_replacement(df_respeck_trimmed['gyro_z'])
#df_respeck_trimmed['accel_x'] = outliner_replacement(df_respeck_trimmed['accel_x'])
#df_respeck_trimmed['accel_y'] = outliner_replacement(df_respeck_trimmed['accel_y'])
#df_respeck_trimmed['accel_z'] = outliner_replacement(df_respeck_trimmed['accel_z'])
#df_thingy_trimmed['accel_x'] = outliner_replacement(df_thingy_trimmed['accel_x'])
#df_thingy_trimmed['accel_y'] = outliner_replacement(df_thingy_trimmed['accel_y'])
#df_thingy_trimmed['accel_z'] = outliner_replacement(df_thingy_trimmed['accel_z'])
#df_thingy_trimmed['gyro_x'] = outliner_replacement(df_thingy_trimmed['gyro_x'])
#df_thingy_trimmed['gyro_y'] = outliner_replacement(df_thingy_trimmed['gyro_y'])
#df_thingy_trimmed['gyro_z'] = outliner_replacement(df_thingy_trimmed['gyro_z'])

In [ ]:
df_respeck_trimmed

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 12))

# plot respeck
ax[0].plot(df_respeck_trimmed['accel_x'], label="accel_x")
ax[0].plot(df_respeck_trimmed['accel_y'], label="accel_y")
ax[0].plot(df_respeck_trimmed['accel_z'], label="accel_z")
ax[0].legend()

ax[0].set_title(f"{df_respeck_trimmed['sensor_type'].values[0]} - {df_respeck_trimmed['activity_type'].values[0]} \n Accelerometer data")

ax[1].plot(df_respeck_trimmed['gyro_x'], label="gyro_x")
ax[1].plot(df_respeck_trimmed['gyro_y'], label="gyro_y")
ax[1].plot(df_respeck_trimmed['gyro_z'], label="gyro_z")
ax[1].legend()

ax[1].set_title(f"{df_respeck_trimmed['sensor_type'].values[0]} - {df_respeck_trimmed['activity_type'].values[0]} \n Gyroscope data")

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(12, 18))

# plot thingy
ax[0].plot(df_thingy_trimmed['accel_x'], label="accel_x")
ax[0].plot(df_thingy_trimmed['accel_y'], label="accel_y")
ax[0].plot(df_thingy_trimmed['accel_z'], label="accel_z")
ax[0].legend()

ax[0].set_title(f"{df_thingy_trimmed['sensor_type'].values[0]} - {df_thingy_trimmed['activity_type'].values[0]} \n Accelerometer data")

ax[1].plot(df_thingy_trimmed['gyro_x'], label="gyro_x")
ax[1].plot(df_thingy_trimmed['gyro_y'], label="gyro_y")
ax[1].plot(df_thingy_trimmed['gyro_z'], label="gyro_z")
ax[1].legend()

ax[1].set_title(f"{df_thingy_trimmed['sensor_type'].values[0]} - {df_thingy_trimmed['activity_type'].values[0]} \n Gyroscope data")

ax[2].plot(df_thingy_trimmed['mag_x'], label="mag_x")
ax[2].plot(df_thingy_trimmed['mag_y'], label="mag_y")
ax[2].plot(df_thingy_trimmed['mag_z'], label="mag_z")
ax[2].legend()

ax[2].set_title(f"{df_thingy_trimmed['sensor_type'].values[0]} - {df_thingy_trimmed['activity_type'].values[0]} \n Magnetometer data")

## Saving your clean data

When you are done cleaning your data you can save it to a location of your choice by running the following commands:

In [6]:
rec_name = df_respeck_trimmed.recording_id.values[0]
print(rec_name)

df_respeck_trimmed.to_csv(f"./Data/Clean/{rec_name}.csv", index=False)

NameError: name 'df_respeck_trimmed' is not defined

In [7]:
rec_name = df_thingy_trimmed.recording_id.values[0]
print(rec_name)

df_thingy_trimmed.to_csv(f"./Data/Clean/{rec_name}.csv", index=False)

NameError: name 'df_thingy_trimmed' is not defined

## Uploading collected data

The format in which you should save each of your recordings is the format we arrive at in this notebook. That is:
- no header
- all the header information transformed into column values
- column list:
    * timestamp
    * accel_x, accel_y, accel_z
    * gyro_x, gyro_y, gyro_z
    * (for Thingy recordings) mag_x, mag_y, mag_z
    * sensor_type
    * activity_type
    * activity_code
    * subject_id
    * notes
    * recording_id
    
Be very vareful when uploading these to the shared repository and make sure your files are in the correct format. We will be checking your submissions automatically. 

You can double check the columns of your dataframe by running:

In [ ]:
df_respeck_trimmed.columns

## Common techniques to consider

* There are two main ways in which you can tackle this HAR task: using Machine Learning algorithms (Random Forest Classifier (RFC), Clustering, Regression etc), or Deep Learning methods (Convolutional Neural Networks (CNN), Recurrent Neural Networks (RNN) etc).


* The most common way to preprocess time series data is to divide it into sliding windows (you can choose how much they overlap).


* The sliding windows can then be directly passed to your algorithm (for example the CNN), or you can extract features of the signal from the windows and pass a vector of features to the classification algorithm (for example, a RFC). 


* Be very careful about splitting the data into training, validation and test sets. Your algorithms will perform extremely well when data is coming from the same subject. You need to test your algorithms with a technique called Leave One Subject Out Cross Validation (LOSOXV) whereby you test your method on data from an unseen subject.

The Week 2 - Introduction to Human Activity Recognition lab gives you an overview of these techniques.